In [ ]:
import matplotlib.pyplot as plt

def draw_arr3d(arr):
    grid = arr.copy()
    grid = grid % 7
    layers = grid.shape[2]
    print(grid.shape)

    fig, axs = plt.subplots(nrows=1, ncols=6, figsize=(20, 6),
                            subplot_kw={'xticks': [], 'yticks': []})

    for ax, t in zip(axs.flat, range(6)):
        ax.imshow(grid[:,:,t], interpolation=None, cmap='viridis')
        ax.set_title('gen ' + str(t))

    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
from math import floor
from scipy.signal import convolve2d

def next_state(alive_n, ambiguous_n, center):
    if center == 2: 
        if alive_n == 3 and ambiguous_n == 0:
            return 1
        if alive_n > 3:
            return 0
        if alive_n + ambiguous_n < 2:
            return 0
        return 2

    neighbours = set(range(alive_n, alive_n + ambiguous_n + 1))
    if center == 0:
        target = {3}
    else:
        target = {2,3}

    if neighbours.isdisjoint(target):
        return 0
    if neighbours <= target:
        return 1
        
    return 2

import numpy as np
from math import floor
from random import randint, random

def gen_noisy_block(start_block, n_layer = 0, corruption_per = 0.03):
    area = start_block.shape[0]*start_block.shape[1]
    to_corrupt = int(area*corruption_per)
    
    new_block_pattern = start_block[:,:,n_layer].copy()
    new_block = np.zeros((16,16), dtype = int)

    dx = randint(0, 16 - start_block.shape[0])
    dy = randint(0, 16 - start_block.shape[1])

    new_block[dx:dx+start_block.shape[0],dy:dy+start_block.shape[1]] = new_block_pattern

    indices = np.random.choice(new_block.shape[1]*new_block.shape[0], replace=False, size=to_corrupt)

    new_block[np.unravel_index(indices, new_block.shape)] = 16 

    if random() < 0.5:
        new_block = np.flip(new_block)
    if random() > 0.25:
        new_block = np.rot90(new_block, k = randint(1, 3))

    full_block = np.zeros((16,16,6), dtype=int)
    full_block[:,:,0] = new_block

    return full_block, to_corrupt

to_small_val = lambda t: 2 if t == 16 else t

def update_ambigous_block(arr):
    from scipy.signal import convolve2d
    tr = arr.shape
    iters = arr.shape[2]

    scharr = np.array([[1, 1, 1],
                       [1, 0, 1],
                       [1, 1, 1]], dtype=int)
    
    for i in range(iters-1):

        arr[:,:,i+1] = convolve2d(arr[:,:,i], scharr, mode='same', boundary='fill', fillvalue=0)
        for x in range(arr.shape[0]):
            for y in range(arr.shape[1]):

                out = next_state(arr[x,y,i+1]%16, arr[x,y,i+1]//16, to_small_val(arr[x,y,i]))
                if out == 2:
                    arr[x,y,i+1] = 16
                else:
                    arr[x,y,i+1] = out 

    return arr 

def has_3_on_boarder(arr):
    tr = arr.shape
    for t in range(tr[2]):
        for j in range(2, tr[0]):
            if arr[j-2, 0, t] == 1 and arr[j-1, 0, t] == 1 and arr[j, 0, t] == 1:
                return True

            if arr[j-2, tr[0]-1, t] == 1 and arr[j-1, tr[0]-1, t] == 1 and arr[j, tr[0]-1, t] == 1:
                return True

            if arr[0, j-2, t] == 1 and arr[0, j-1, t] == 1 and arr[0, j, t] == 1:
                return True

            if arr[tr[0]-1, j-2, t] == 1 and arr[tr[0]-1, j-1, t] == 1 and arr[tr[0]-1, j, t] == 1:
                return True
    return False


    

In [ ]:
test_block = np.zeros((9, 9, 6), dtype = np.uint8)
test_block[5,5,0] = 1
test_block[5,6,0] = 1
test_block[5,7,0] = 1
test_block[6,5,0] = 16


draw_arr3d(test_block)
k = update_ambigous_block(test_block)

draw_arr3d(k)

In [ ]:
test_b1 = np.array([[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], [[1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], [[1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], [[0, 1, 1, 1, 1, 0], [1, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1]], [[0, 0, 1, 1, 0, 0], [1, 1, 1, 0, 0, 1], [1, 0, 1, 1, 0, 1], [1, 1, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]], [[0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 0], [1, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 0, 1, 1, 1, 1], [1, 0, 1, 1, 0, 1], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 0], [1, 1, 1, 0, 0, 1], [1, 0, 0, 0, 0, 1], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 0, 0], [0, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0]]])

t, n_corrupted = gen_noisy_block(test_b1, 0, 0.2)
print(n_corrupted)
draw_arr3d(t)
draw_arr3d(update_ambigous_block(t))


NameError: ignored

In [ ]:
import gym
from gym import spaces

size_bbox = 16
period = 5
N_DISCRETE_ACTIONS = size_bbox*size_bbox*2

class CustomEnv(gym.Env):
    """Custom Environment that follows gym interface"""

    metadata = {"render.modes": ["human"]}

    def __init__(self):
        super(CustomEnv, self).__init__()

        self.action_space = spaces.Discrete(N_DISCRETE_ACTIONS)
        self.observation_space = spaces.Box(low=0, high=2, shape=(period+1, size_bbox, size_bbox), dtype=np.uint8)

        self._action_to_coords = lambda t: (t // (size_bbox*size_bbox),
                                            (t%(size_bbox*size_bbox)) //size_bbox,
                                            (t%(size_bbox*size_bbox)) % size_bbox)
        #self.state = 

    def step(self, action):
        self.state += action -1 
        self.shower_length -= 1 
        
        if self.state >=37 and self.state <=39: 
            reward = 1 
        else: 
            reward = -1 
        
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        info = {}
        
        # Return step information
        return self.state, reward, done, info
    
    def reset(self, corruption_level = 0.1):

        return self.state